# Exercise Sheet 1

## Exercise 1-1 Linear Algebra

Let $a = (1,2,1)^T$ and $b = (2,2,1)^T$ be vectors and let
$A = \begin{pmatrix} 1 & 2 \\ 2 & 1 \\ 1 & 1 \end{pmatrix}$, $B = \begin{pmatrix} 2 & 1 \\ 1 & 2 \end{pmatrix}$, $C = \begin{pmatrix} 1 & 3 \\ 0 & 1 \end{pmatrix}$

In [10]:
import numpy as np
import matplotlib.pyplot as plt

a = np.mat([[1], [2], [1]])
b = np.mat([[2], [2], [1]])

A = np.mat([[1, 2], [2, 1], [1, 1]])
B = np.mat([[2, 1], [1, 2]])
C = np.mat([[1, 3], [0, 1]])

## a)
Calculate the following results (either with pen and paper or a programming language of your choice)

In [12]:
print 'a^T b = '
print a.T * b

print 'a b^T = '
print a * b.T

print 'A C ='
print A * C

print 'C A^T = '
print C * A.T

print 'A^T a = '
print A.T * a

print 'a^T A = '
print a.T * A

a^T b = 
[[7]]
a b^T = 
[[2 2 1]
 [4 4 2]
 [2 2 1]]
A C =
[[1 5]
 [2 7]
 [1 4]]
C A^T = 
[[7 5 4]
 [2 1 1]]
A^T a = 
[[6]
 [5]]
a^T A = 
[[6 5]]


## b)
Invert B and check if $B^{-1}B = BB^{-1} = I$ holds.

In [19]:
print 'B^-1 = '
print B.I

print 'B^-1 * B = '
print B.I * B

print 'B * B^-1 = '
print B * B.I

print 'I = '
print np.identity(2)

print 'B^-1 * B = I: '
print (B.I * B == np.identity(2)).all()

print 'B * B^-1 = I: '
print (B * B.I == np.identity(2)).all()

B^-1 = 
[[ 0.66666667 -0.33333333]
 [-0.33333333  0.66666667]]
B^-1 * B = 
[[ 1.  0.]
 [ 0.  1.]]
B * B^-1 = 
[[ 1.  0.]
 [ 0.  1.]]
I = 
[[ 1.  0.]
 [ 0.  1.]]
B^-1 * B = I: 
True
B * B^-1 = I: 
True


## c)
Generate an orthonormal 3 × 3 matrix. Check if rows and columns are indeed orthonormal.

In [16]:
#??????????????????????
orthonormal = alle Spalten sind orthogonal

## Exercise 1-2 Recap: Vector Calculus

Compute $\frac{\partial g(x)}{\partial x}$ the function below.
_Hint_: For a function $g(x):\mathbb{R}^n \rightarrow \mathbb{R}$ miti $x \in \mathbb{R}^n$ holds:

$$\frac{\partial g(x)}{\partial x} = \begin{bmatrix} \frac{\partial g(x)}{\partial x_1} \\ \frac{\partial g(x)}{\partial x_2} \\ \vdots \\\frac{\partial g(x)}{\partial x_n} \end{bmatrix} $$.

## a)
$g(x) = \sum^n_{i=1}x_i$

$\frac{\partial g(x)}{\partial x} = \begin{pmatrix} 1 \\ 1 \\ \vdots \\ 1 \end{pmatrix}$

## b)
$g(x) = (x,x)$, the standard scalar product of x with itself

$\frac{\partial g(x)}{\partial x} = \begin{pmatrix} 2x_1 \\ 2x_2 \\ \vdots \\ 2x_n\end{pmatrix}$

## c)
$g(x) = (x-\mu)^2$ für $\mu \in \mathbb{R}^n$

#### Setup

In [2]:
import itertools
import random

import numpy as np

In [3]:
patterns = [np.matrix([[x], [y]]) for x, y in [(2,4), (1,0.5), (0.5,1.5), (0,0.5)]]
labels = [1, 1, -1, -1]

In [4]:
def get_shuffled(labels, patterns):
    zipped = list(zip(labels, patterns))
    random.shuffle(zipped)
    labels_shuf, patterns_shuf = list(zip(*zipped))
    return labels_shuf, patterns_shuf

#### a) How many iterations are required by the pattern-based perceptron learning rule in order to seperate classes A and B correctly if the weight vector w is initialized as (0, 1, −1) and step size η is set to 0.1?

In [5]:
from scipy.special import expit as sigmoid

class Perceptron:
    def __init__(self, initial_weights):
        self._initial_weights = initial_weights
        self.weights = initial_weights[:]
        
    def _add_bias(self, data):
            return np.vstack([[1], data])
    
    def reset(self):
        self.weights = self._initial_weights[:]
        
    def predict(self, data):
        if len(self.weights) - len(data) == 1:
            data = self._add_bias(data)
        return np.sign(sum(w*data[j] for j, w in enumerate(self.weights)))
    
    def fit(self, data, classes, learn_rate, max_iterations):
        raise NotImplemented()
    
    
class PatternPerceptron(Perceptron):
    def fit(self, data, classes, learn_rate=0.1, max_iterations=100):
        # Add bias to data
        data = [np.vstack([[1], d]) for d in data]
        def has_converged():
            return all(self.predict(x) == y
                                    for x, y in zip(data, classes))
        iterations = 0
        while iterations < max_iterations and not has_converged():
            iterations += 1
            # Error is here, we must not take *all* data into account, but merely a single one
            # that we pick sequentially
            # => Sample-based learning!
            currently_misclassified = [(x, y) for x, y in zip(data, classes)
                                       if self.predict(x) != y]
            for j in range(len(self.weights)):
                increase = learn_rate * sum(y*x[j] for x, y in currently_misclassified)
                self.weights[j] += increase
        if not has_converged():
            raise Exception("Did not converge after {} iterations"
                            .format(iterations))
        return iterations

In [6]:
p = PatternPerceptron(initial_weights=[0, 1, -1])
print("Answer: {} iterations are required.".format(
    p.fit(patterns, labels, learn_rate=0.1, max_iterations=2000)))

Answer: 1 iterations are required.


CORRECT

#### b) How many iterations are required if η = 0.25?    Is the order of the considered patterns relevant? If so, give an example, otherwise, prove it.

In [7]:
p.reset()
print("Answer: {} iterations are required with η = 0.25.".format(
    p.fit(patterns, labels, learn_rate=0.25, max_iterations=100)))

randomized_iters = []
for _ in range(100):
    p.reset()
    labels_shuf, patterns_shuf = get_shuffled(labels, patterns)
    randomized_iters.append(
        p.fit(patterns_shuf, labels_shuf, learn_rate=0.25,
              max_iterations=100))
print("Answer: With the patterns in shuffled order, we needed {} iterations, so "
      "the order does not seem to be relevant.".format(set(randomized_iters)))

Answer: 2 iterations are required with η = 0.25.
Answer: With the patterns in shuffled order, we needed set([2]) iterations, so the order does not seem to be relevant.


NOT CORRECT -> Should be 6 iterations
NOT CORRECT -> Does matter
??? How to prove that this must be the case ???

#### c) After how many iterations does the gradient-based learning rule terminate for both η? In this case: Is the order of the considered patterns relevant?

In [8]:
class GradientPerceptron(Perceptron):
    def fit(self, data, classes, learn_rate=0.1, max_iterations=100):
        # Add bias to data
        data = [np.vstack([[1], d]) for d in data]
        has_converged = lambda: all(self.predict(x) == y
                                    for x, y in zip(data, classes))
        iterations = 0
        while iterations < max_iterations and not has_converged():
            iterations += 1
            # ERROR: According to the solution, we should go over *all* misclassified patterns??
            i = np.random.randint(len(data))
            # We only want to select misclassified patterns
            if classes[i] == self.predict(data[i]):
                continue
            for j in range(len(self.weights)):
                increase = learn_rate * classes[i] * data[i][j]
                self.weights[j] += increase
        if not has_converged():
            raise Exception("Did not converge after {} iterations"
                            .format(iterations))
        return iterations

In [9]:
gp = GradientPerceptron(initial_weights=[0, 1, -1])
for rate in (0.1, 0.25):
    gp.reset()
    gp.fit(patterns, labels, learn_rate=rate, max_iterations=100)
    print("Answer: With a rate of {}, gradient-based learning terminates.".format(rate))

for rate in (0.1, 0.25):
    for _ in range(100):
        gp.reset()
        labels_shuf, patterns_shuf = get_shuffled(labels, patterns)
        gp.fit(patterns_shuf, labels_shuf, learn_rate=0.1,
              max_iterations=100)
    print("With a rate of {} and shuffled data and labels it seems to terminate as well.".format(rate))


Answer: With a rate of 0.1, gradient-based learning terminates.
Answer: With a rate of 0.25, gradient-based learning terminates.
With a rate of 0.1 and shuffled data and labels it seems to terminate as well.
With a rate of 0.25 and shuffled data and labels it seems to terminate as well.
